## Importing all necessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

import requests # library to handle requests

print('Libraries imported.')

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-main==custom=2020
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2020
done

# All requested packages already installed.

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-main==custom=2020
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2020
done

# All requested packages already installed.

Libraries imported.


##  Accessing the GeoJSON file that contains the neighborhoods of Zurich

In [2]:
# The code was removed by Watson Studio for sharing.

##  Reading the GeoJSON file into a pandas Dataframe & extracting features

In [3]:
zurich_data = pd.read_json(body, orient='values')
zurich_data.head()

,type,name,features
0,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
1,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
2,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
3,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
4,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."


In [4]:
zurich_data_features = zurich_data['features']
zurich_data_features.head()

0    {'type': 'Feature', 'geometry': {'type': 'Poin...
1    {'type': 'Feature', 'geometry': {'type': 'Poin...
2    {'type': 'Feature', 'geometry': {'type': 'Poin...
3    {'type': 'Feature', 'geometry': {'type': 'Poin...
4    {'type': 'Feature', 'geometry': {'type': 'Poin...
Name: features, dtype: object

## Put together information about Zurich neighborhoods into a dataframe

In [5]:
# Define the dataframe columns for a neighborhoods dataframe
column_names = ['Borough','Neighborhood', 'Latitude', 'Longitude'] 

# Instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [6]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [7]:
# Extract necessary neighborhood information into the dataframe: name, latitude, longitude
for data in zurich_data_features:
    neighborhood_name = data['properties']['name'] 

    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': "Zürich",
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, 
                                           ignore_index=True)
    

In [8]:
neighborhoods.shape

(22, 4)

In [9]:
# How many neighborhoods of Zurich did we get
print('The dataframe has {} neighborhoods.'.format(
        len(neighborhoods['Neighborhood'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 22 neighborhoods.


In [10]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude
0,Zürich,Hirslanden,47.363057,8.570403
1,Zürich,Witikon,47.359977,8.600887
2,Zürich,Leimbach,47.331480,8.509313
3,Zürich,Wollishofen,47.341491,8.532956
4,Zürich,Enge,47.360931,8.533140
5,Zürich,Riesbach,47.354570,8.556241
6,Zürich,Affoltern,47.423281,8.509221
7,Zürich,Seebach,47.423695,8.542830
8,Zürich,Oerlikon,47.409348,8.545117
9,Zürich,Höngg,47.407904,8.496424


## Visualize the neighborhoods of Zurich on the map

In [11]:
# Get the geographical cooordinates of Zurich
address = 'Zurich, Zurich'

geolocator = Nominatim(user_agent="zh_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Zurich are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Zurich are 47.3744489, 8.5410422.


In [12]:
# create map of Zurich using latitude and longitude values
map_zh = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood, borough in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood'], neighborhoods['Borough']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zh)  
    
map_zh

## Foursquare Credentials and Version

In [13]:
CLIENT_ID = 'D0WVJEV1OWOCGMZ23LW3AV2PUIHF2QXIWCLWQS0V0NBMWV0Q' # your Foursquare ID
CLIENT_SECRET = 'F54CW4QHMTFO23ZZ0LTCEC4XU42PVEEADHNJCNPNDTKVVXIQ' # your Foursquare Secret
ACCESS_TOKEN = 'EUXOMD4V042AJ0VBANQVMRQDPSALDTJLWFD4CVMNIMNYIPVE' # Foursquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: D0WVJEV1OWOCGMZ23LW3AV2PUIHF2QXIWCLWQS0V0NBMWV0Q
CLIENT_SECRET:F54CW4QHMTFO23ZZ0LTCEC4XU42PVEEADHNJCNPNDTKVVXIQ


## Explore neighborhoods in Zurich

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
zurich_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                latitudes=neighborhoods['Latitude'],
                                longitudes=neighborhoods['Longitude']
                                )

Hirslanden
Witikon
Leimbach
Wollishofen
Enge
Riesbach
Affoltern
Seebach
Oerlikon
Höngg
Unterstrass
Wipkingen
Schwamendingen
Oberstrass
Altstetten
Aussersihl
Altstadt
Fluntern
Hottingen
Albisrieden
Wiedikon
Industriequartier


## How many venues in the Zurich neighborhoods did Foursquare return

In [16]:
print(zurich_venues.shape)
zurich_venues.head()

(520, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hirslanden,47.363057,8.570403,"Marcellino, Pane e Vino",47.360202,8.565569,Mediterranean Restaurant
1,Hirslanden,47.363057,8.570403,Wilder Mann,47.360341,8.566796,Swiss Restaurant
2,Hirslanden,47.363057,8.570403,Ristorante Antiquario,47.360676,8.566550,Italian Restaurant
3,Hirslanden,47.363057,8.570403,Klusplatz,47.364025,8.566751,Plaza
4,Hirslanden,47.363057,8.570403,Bistro Le Puy,47.359382,8.568897,French Restaurant


In [17]:
# Check unique categories we received for Zurich venues
print('There are {} unique categories.'.format(len(zurich_venues['Venue Category'].unique())))

There are 142 unique categories.


## Lets check which Zurich neighborhoods have the highest number of venues 

In [18]:
# Sort and print the total list of zurich neighborhoods
zurich_venues.groupby('Neighborhood').count().sort_values(by=['Venue'], ascending=False)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Industriequartier,100,100,100,100,100,100
Altstadt,87,87,87,87,87,87
Aussersihl,64,64,64,64,64,64
Oerlikon,42,42,42,42,42,42
Riesbach,41,41,41,41,41,41
Enge,31,31,31,31,31,31
Altstetten,26,26,26,26,26,26
Unterstrass,20,20,20,20,20,20
Wipkingen,18,18,18,18,18,18


## Pick the venue categories of interest, according to the Business Problem: transportation, food places, stores

In [19]:
# Check the venue categories 
zurich_venues['Venue Category'].unique()

array(['Mediterranean Restaurant', 'Swiss Restaurant',
       'Italian Restaurant', 'Plaza', 'French Restaurant', 'Tram Station',
       'Park', 'Hotel', 'Supermarket', 'Bus Station', 'Soccer Field',
       'Grocery Store', 'Tennis Court', 'Moroccan Restaurant',
       'Restaurant', 'Music Venue', 'Cheese Shop', 'Community Center',
       'Irish Pub', 'Pool', 'Salon / Barbershop', 'Fast Food Restaurant',
       'Gas Station', 'History Museum', 'BBQ Joint', 'Sauna / Steam Room',
       'Thai Restaurant', 'Burger Joint', 'Bar', 'Train Station',
       'Middle Eastern Restaurant', 'Chinese Restaurant', 'Cupcake Shop',
       'Coffee Shop', 'Japanese Restaurant', 'Beach', 'Movie Theater',
       'Café', 'Garden', 'Taverna', 'Seafood Restaurant', 'Bakery',
       'Art Museum', 'Museum', 'Mexican Restaurant', 'Beer Garden',
       'Wine Bar', 'Playground', 'Sushi Restaurant',
       'Arts & Crafts Store', 'Food Court', 'Snack Place',
       'Department Store', 'Athletics & Sports', 'Diner', 

## Considering public transportation in Zurich neighborhoods 

In [20]:
# We select all entries with public transportation (train stations, tram stations, bus stations)   
target=zurich_venues.loc[zurich_venues['Venue Category'].isin(['Train Station', 'Tram Station','Bus Station'])]   
target

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
5,Hirslanden,47.363057,8.570403,VBZ Klusplatz,47.364090,8.566398,Tram Station
9,Witikon,47.359977,8.600887,Glockenacker,47.360889,8.598892,Bus Station
12,Witikon,47.359977,8.600887,VBZ Loorenstrasse,47.359903,8.594463,Bus Station
25,Wollishofen,47.341491,8.532956,VBZ Rote Fabrik,47.342868,8.536043,Bus Station
26,Wollishofen,47.341491,8.532956,VBZ Morgental,47.343597,8.530199,Tram Station
30,Wollishofen,47.341491,8.532956,VBZ Seerose,47.339308,8.537885,Bus Station
52,Enge,47.360931,8.533140,Bahnhof Zürich Enge,47.364305,8.530746,Train Station
57,Enge,47.360931,8.533140,VBZ Bahnhof Enge / Bederstrasse,47.364629,8.530435,Tram Station
97,Riesbach,47.354570,8.556241,VBZ Fröhlichstrasse,47.353632,8.556770,Tram Station
104,Affoltern,47.423281,8.509221,Bahnhof Zürich Affoltern,47.420950,8.507619,Train Station


In [21]:
target.shape

(39, 7)

In [22]:
# We introduce a 'Venue Group' which puts the relevant entries together 
# In this case venue categories 'Train Station', 'Bus Station' and 'Tram Station' are put together to Venue Group "Public Transportation"
target.insert(7, 'Venue Group', "Public Transportation", True)
target.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Group
5,Hirslanden,47.363057,8.570403,VBZ Klusplatz,47.364090,8.566398,Tram Station,Public Transportation
9,Witikon,47.359977,8.600887,Glockenacker,47.360889,8.598892,Bus Station,Public Transportation
12,Witikon,47.359977,8.600887,VBZ Loorenstrasse,47.359903,8.594463,Bus Station,Public Transportation
25,Wollishofen,47.341491,8.532956,VBZ Rote Fabrik,47.342868,8.536043,Bus Station,Public Transportation
26,Wollishofen,47.341491,8.532956,VBZ Morgental,47.343597,8.530199,Tram Station,Public Transportation


In [23]:
#target.groupby(["Neighborhood"]).count().sort_values(by=['Venue Category'], ascending=False)

## Selecting Zurich neighborhoods with the highest number of public transportation

In [24]:
target.value_counts(['Venue Group', 'Neighborhood']).reset_index(name='count').sort_values(by=['Venue Group','count'], ascending=False)

,Venue Group,Neighborhood,count
0,Public Transportation,Schwamendingen,5
1,Public Transportation,Wiedikon,4
2,Public Transportation,Unterstrass,4
3,Public Transportation,Oerlikon,3
4,Public Transportation,Wipkingen,3
5,Public Transportation,Wollishofen,3
6,Public Transportation,Albisrieden,2
7,Public Transportation,Enge,2
8,Public Transportation,Seebach,2
9,Public Transportation,Aussersihl,2


### -> It seems that Schwamedingen, Wiedikon and Unterstrass have highest number of public transportation.

## Considering Zurich neighborhoods with the highest number of food places

In [25]:
# Now lets add food places to the target dataframe
target = target.append(zurich_venues.loc[zurich_venues['Venue Category'].str.contains("Restaurant")])
target = target.append(zurich_venues.loc[zurich_venues['Venue Category'].str.contains("Café")])
target.loc[target['Venue Group'].isnull(), 'Venue Group'] = 'Food places'
target

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Group
5,Hirslanden,47.363057,8.570403,VBZ Klusplatz,47.364090,8.566398,Tram Station,Public Transportation
9,Witikon,47.359977,8.600887,Glockenacker,47.360889,8.598892,Bus Station,Public Transportation
12,Witikon,47.359977,8.600887,VBZ Loorenstrasse,47.359903,8.594463,Bus Station,Public Transportation
25,Wollishofen,47.341491,8.532956,VBZ Rote Fabrik,47.342868,8.536043,Bus Station,Public Transportation
26,Wollishofen,47.341491,8.532956,VBZ Morgental,47.343597,8.530199,Tram Station,Public Transportation
30,Wollishofen,47.341491,8.532956,VBZ Seerose,47.339308,8.537885,Bus Station,Public Transportation
52,Enge,47.360931,8.533140,Bahnhof Zürich Enge,47.364305,8.530746,Train Station,Public Transportation
57,Enge,47.360931,8.533140,VBZ Bahnhof Enge / Bederstrasse,47.364629,8.530435,Tram Station,Public Transportation
97,Riesbach,47.354570,8.556241,VBZ Fröhlichstrasse,47.353632,8.556770,Tram Station,Public Transportation
104,Affoltern,47.423281,8.509221,Bahnhof Zürich Affoltern,47.420950,8.507619,Train Station,Public Transportation


In [26]:
target.shape

(201, 8)

In [27]:
#target.groupby(["Neighborhood"]).count().sort_values(by=['Venue Category'], ascending=False)

## Selecting Zurich neighborhoods with the highest number of food places

In [28]:

target.value_counts(['Venue Group', 'Neighborhood']).reset_index(name='count').sort_values(by=['Venue Group','count'], ascending=False)

,Venue Group,Neighborhood,count
9,Public Transportation,Schwamendingen,5
11,Public Transportation,Wiedikon,4
12,Public Transportation,Unterstrass,4
14,Public Transportation,Wipkingen,3
15,Public Transportation,Oerlikon,3
16,Public Transportation,Wollishofen,3
17,Public Transportation,Aussersihl,2
18,Public Transportation,Enge,2
20,Public Transportation,Seebach,2
21,Public Transportation,Witikon,2


### -> It seems that Industriequartier, Aussersihl, Altstadt and Oerlikon have the highest number of food places. 

## And last but not least, considering supermarkets and stores in each neighborhood

In [29]:
# Lets add supermarkets and stores to the target dataframe

target = target.append(zurich_venues.loc[zurich_venues['Venue Category'].str.contains("Shop")])
target = target.append(zurich_venues.loc[zurich_venues['Venue Category'].str.contains("Store")])
target = target.append(zurich_venues.loc[zurich_venues['Venue Category'].str.contains("Supermarket")])
target.loc[target['Venue Group'].isnull(), 'Venue Group'] = 'Stores'
target

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Group
5,Hirslanden,47.363057,8.570403,VBZ Klusplatz,47.364090,8.566398,Tram Station,Public Transportation
9,Witikon,47.359977,8.600887,Glockenacker,47.360889,8.598892,Bus Station,Public Transportation
12,Witikon,47.359977,8.600887,VBZ Loorenstrasse,47.359903,8.594463,Bus Station,Public Transportation
25,Wollishofen,47.341491,8.532956,VBZ Rote Fabrik,47.342868,8.536043,Bus Station,Public Transportation
26,Wollishofen,47.341491,8.532956,VBZ Morgental,47.343597,8.530199,Tram Station,Public Transportation
30,Wollishofen,47.341491,8.532956,VBZ Seerose,47.339308,8.537885,Bus Station,Public Transportation
52,Enge,47.360931,8.533140,Bahnhof Zürich Enge,47.364305,8.530746,Train Station,Public Transportation
57,Enge,47.360931,8.533140,VBZ Bahnhof Enge / Bederstrasse,47.364629,8.530435,Tram Station,Public Transportation
97,Riesbach,47.354570,8.556241,VBZ Fröhlichstrasse,47.353632,8.556770,Tram Station,Public Transportation
104,Affoltern,47.423281,8.509221,Bahnhof Zürich Affoltern,47.420950,8.507619,Train Station,Public Transportation


In [30]:
target.shape

(282, 8)

## Selecting Zurich neighborhoods with the highest number of stores

In [31]:
target.value_counts(['Venue Group', 'Neighborhood']).reset_index(name='count').sort_values(by=['Venue Group','count'], ascending=False)

,Venue Group,Neighborhood,count
3,Stores,Altstadt,24
5,Stores,Industriequartier,13
10,Stores,Riesbach,6
11,Stores,Oerlikon,5
12,Stores,Altstetten,5
14,Stores,Wipkingen,5
17,Stores,Unterstrass,4
18,Stores,Enge,4
24,Stores,Wollishofen,3
27,Stores,Witikon,2


### -> It seems that Altstadt and Industriequartier have the highest number of stores

## Discovering which neighborhood has a highest score in total

In [32]:
target.insert(8, 'Score', 1, True)
target

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Group,Score
5,Hirslanden,47.363057,8.570403,VBZ Klusplatz,47.364090,8.566398,Tram Station,Public Transportation,1
9,Witikon,47.359977,8.600887,Glockenacker,47.360889,8.598892,Bus Station,Public Transportation,1
12,Witikon,47.359977,8.600887,VBZ Loorenstrasse,47.359903,8.594463,Bus Station,Public Transportation,1
25,Wollishofen,47.341491,8.532956,VBZ Rote Fabrik,47.342868,8.536043,Bus Station,Public Transportation,1
26,Wollishofen,47.341491,8.532956,VBZ Morgental,47.343597,8.530199,Tram Station,Public Transportation,1
30,Wollishofen,47.341491,8.532956,VBZ Seerose,47.339308,8.537885,Bus Station,Public Transportation,1
52,Enge,47.360931,8.533140,Bahnhof Zürich Enge,47.364305,8.530746,Train Station,Public Transportation,1
57,Enge,47.360931,8.533140,VBZ Bahnhof Enge / Bederstrasse,47.364629,8.530435,Tram Station,Public Transportation,1
97,Riesbach,47.354570,8.556241,VBZ Fröhlichstrasse,47.353632,8.556770,Tram Station,Public Transportation,1
104,Affoltern,47.423281,8.509221,Bahnhof Zürich Affoltern,47.420950,8.507619,Train Station,Public Transportation,1


In [33]:
# As public transportation is very important, we give it a triple weight
target.loc[target['Venue Group'] == 'Public Transportation', 'Score'] = 3

# As proximity to stores and supermarkets is also important, we give it a double weight
target.loc[target['Venue Group'] == 'Stores', 'Score'] = 2
target

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Group,Score
5,Hirslanden,47.363057,8.570403,VBZ Klusplatz,47.364090,8.566398,Tram Station,Public Transportation,3
9,Witikon,47.359977,8.600887,Glockenacker,47.360889,8.598892,Bus Station,Public Transportation,3
12,Witikon,47.359977,8.600887,VBZ Loorenstrasse,47.359903,8.594463,Bus Station,Public Transportation,3
25,Wollishofen,47.341491,8.532956,VBZ Rote Fabrik,47.342868,8.536043,Bus Station,Public Transportation,3
26,Wollishofen,47.341491,8.532956,VBZ Morgental,47.343597,8.530199,Tram Station,Public Transportation,3
30,Wollishofen,47.341491,8.532956,VBZ Seerose,47.339308,8.537885,Bus Station,Public Transportation,3
52,Enge,47.360931,8.533140,Bahnhof Zürich Enge,47.364305,8.530746,Train Station,Public Transportation,3
57,Enge,47.360931,8.533140,VBZ Bahnhof Enge / Bederstrasse,47.364629,8.530435,Tram Station,Public Transportation,3
97,Riesbach,47.354570,8.556241,VBZ Fröhlichstrasse,47.353632,8.556770,Tram Station,Public Transportation,3
104,Affoltern,47.423281,8.509221,Bahnhof Zürich Affoltern,47.420950,8.507619,Train Station,Public Transportation,3


In [34]:
# Get the Total Score per neighborhood
target.groupby(['Neighborhood'], as_index=False)['Score'].sum().sort_values(by=['Score'], ascending=False)

,Neighborhood,Score
2,Altstadt,75
10,Industriequartier,59
4,Aussersihl,38
12,Oerlikon,37
13,Riesbach,28
14,Schwamendingen,27
16,Unterstrass,25
5,Enge,23
3,Altstetten,22
18,Wipkingen,21


In [35]:
#target.value_counts(['Neighborhood']).reset_index(name='count').sort_values(by=['count'], ascending=False)

### -> The result is: Altstadt (= in German "Old city") collected the highest score with regard to relevant criteria. However, the rent prices in Old City might be very high. So we will also be taking into consideration Industriequartier (=In German "Industrial area")


In [36]:
# one hot encoding
zurich_onehot = pd.get_dummies(zurich_venues[['Venue Category']], prefix="", prefix_sep="")
#zurich_onehot = pd.get_dummies(neigh_with_train[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
zurich_onehot['Neighborhood'] = zurich_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [zurich_onehot.columns[-1]] + list(zurich_onehot.columns[:-1])
zurich_onehot = zurich_onehot[fixed_columns]

zurich_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bakery,Bar,Basketball Court,Beach,Beer Bar,Beer Garden,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Burger Joint,Burrito Place,Bus Station,Business Service,Café,Cambodian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Concert Hall,Convenience Store,Cupcake Shop,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Doner Restaurant,Eastern European Restaurant,Escape Room,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Fondue Restaurant,Food,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Kebab Restaurant,Kids Store,Korean Restaurant,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paella Restaurant,Park,Pedestrian Plaza,Peruvian Restaurant,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spanish Restaurant,Sports Bar,Sports Club,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Swiss Restaurant,Taco Place,Taverna,Tennis Court,Thai Restaurant,Theater,Tibetan Restaurant,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Watch Shop,Water Park,Wine Bar
0,Hirslanden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hirslanden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hirslanden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hirslanden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Hirslanden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
zurich_onehot.shape

(520, 143)

In [38]:
zurich_onehot

,Neighborhood,Accessories Store,American Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bakery,Bar,Basketball Court,Beach,Beer Bar,Beer Garden,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Burger Joint,Burrito Place,Bus Station,Business Service,Café,Cambodian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Concert Hall,Convenience Store,Cupcake Shop,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Doner Restaurant,Eastern European Restaurant,Escape Room,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Fondue Restaurant,Food,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Kebab Restaurant,Kids Store,Korean Restaurant,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paella Restaurant,Park,Pedestrian Plaza,Peruvian Restaurant,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spanish Restaurant,Sports Bar,Sports Club,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Swiss Restaurant,Taco Place,Taverna,Tennis Court,Thai Restaurant,Theater,Tibetan Restaurant,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Watch Shop,Water Park,Wine Bar
0,Hirslanden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hirslanden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hirslanden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hirslanden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Hirslanden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Hirslanden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
6,Hirslanden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,

In [39]:
# Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
zurich_grouped = zurich_onehot.groupby('Neighborhood').mean().reset_index()
zurich_grouped

,Neighborhood,Accessories Store,American Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bakery,Bar,Basketball Court,Beach,Beer Bar,Beer Garden,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Burger Joint,Burrito Place,Bus Station,Business Service,Café,Cambodian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Concert Hall,Convenience Store,Cupcake Shop,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Doner Restaurant,Eastern European Restaurant,Escape Room,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Fondue Restaurant,Food,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Kebab Restaurant,Kids Store,Korean Restaurant,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paella Restaurant,Park,Pedestrian Plaza,Peruvian Restaurant,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spanish Restaurant,Sports Bar,Sports Club,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Swiss Restaurant,Taco Place,Taverna,Tennis Court,Thai Restaurant,Theater,Tibetan Restaurant,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Watch Shop,Water Park,Wine Bar
0,Affoltern,0.00,0.00000,0.00000,0.000000,0.000000,0.200000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.200000,0.000000,0.000000,0.200000,0.000000,0.00,0.00,0.00000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.00000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.00000,0.0,0.00,0.000000,0.000000,0.00,0.00000,0.200000,0.00,0.00000,0.000000,0.00,0.00000,0.0,0.000000,0.00000,0.000000,0.000000,0.200000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
1,Albisrieden,0.00,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.200000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.00000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.20000,0.000000,0.000000,0

In [40]:
zurich_grouped.shape

(22, 143)

In [41]:
num_top_venues = 5

for hood in zurich_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = zurich_grouped[zurich_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Affoltern----
                venue  freq
0    Department Store   0.2
1  Athletics & Sports   0.2
2         Supermarket   0.2
3       Train Station   0.2
4               Diner   0.2


----Albisrieden----
           venue  freq
0    Bus Station   0.2
1  Grocery Store   0.2
2   Tram Station   0.2
3     Playground   0.2
4         Bakery   0.2


----Altstadt----
              venue  freq
0               Bar  0.08
1              Café  0.07
2  Swiss Restaurant  0.07
3             Hotel  0.06
4             Plaza  0.06


----Altstetten----
              venue  freq
0            Bakery  0.08
1       Supermarket  0.08
2             Hotel  0.08
3  Swiss Restaurant  0.08
4              Food  0.04


----Aussersihl----
                venue  freq
0                 Bar  0.17
1    Swiss Restaurant  0.09
2                Café  0.06
3          Restaurant  0.06
4  Italian Restaurant  0.05


----Enge----
                venue  freq
0                Park  0.16
1          Restaurant  0.10
2  Italian Res